In [229]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [230]:
stats = pd.read_csv('player_mvp_stats.csv')
stats

,Unnamed: 0,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,Pts Won,Pts Max,Share,W,L,W/L%,GB,PS/G,PA/G,SRS
0,0,A.C. Green,27,los angeles lakers,PF,82,21,26.4,3.1,6.6,...,0.0,0.0,0.0,58,24,0.707,5.0,106.3,99.6,6.73
1,1,A.C. Green,28,los angeles lakers,PF,82,53,35.4,4.7,9.8,...,0.0,0.0,0.0,43,39,0.524,14.0,100.4,101.5,-0.95
2,2,A.C. Green,29,los angeles lakers,PF,82,55,34.4,4.6,8.6,...,0.0,0.0,0.0,39,43,0.476,23.0,104.2,105.5,-1.20
3,3,A.C. Green,30,phoenix suns,PF,82,55,34.5,5.7,11.3,...,0.0,0.0,0.0,56,26,0.683,7.0,108.2,103.4,4.68
4,4,A.C. Green,31,phoenix suns,SF,82,52,32.8,3.8,7.5,...,0.0,0.0,0.0,59,23,0.720,0.0,110.6,106.8,3.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13972,13972,Žarko Čabarkapa,24,golden state warriors,PF,61,0,8.3,1.1,2.9,...,0.0,0.0,0.0,34,48,0.415,20.0,98.5,99.8,-1.11
13973,13973,Željko Rebrača,29,detroit pistons,C,74,4,15.9,2.6,5.1,...,0.0,0.0,0.0,50,32,0.610,0.0,94.3,92.2,1.69
13974,13974,Željko Rebrača,30,detroit pistons,C,30,12,16.3,2.7,4.8,...,0.0,0.0,0.0,50,32,0.610,0.0,91.4,87.7,2.97
13975,13975,Željko Rebrača,32,los angeles clippers,C,58,2,16.0,2.3,4.0,...,0.0,0.0,0.0,37,45,0.451,25.0,95.7,96.5,-0.47


In [231]:
del stats["Unnamed: 0"]

In [232]:
stats.isnull().sum()

Player        0
Age           0
Team          0
Pos           0
G             0
GS            0
MP            0
FG            0
FGA           0
FG%          65
3P            0
3PA           0
3P%        1902
2P            0
2PA           0
2P%         109
eFG%         65
FT            0
FTA           0
FT%         539
ORB           0
DRB           0
TRB           0
AST           0
STL           0
BLK           0
TOV           0
PF            0
PTS           0
Year          0
Pts Won       0
Pts Max       0
Share         0
W             0
L             0
W/L%          0
GB            0
PS/G          0
PA/G          0
SRS           0
dtype: int64

In [233]:
stats[pd.isnull(stats["3P%"])][["Player", "3PA"]]

,Player,3PA
10,A.J. Bramlett,0.0
46,Aaron Gray,0.0
47,Aaron Gray,0.0
48,Aaron Gray,0.0
69,Aaron McKie,0.0
...,...,...
13970,Žan Tabak,0.0
13973,Željko Rebrača,0.0
13974,Željko Rebrača,0.0
13975,Željko Rebrača,0.0


In [234]:
stats[pd.isnull(stats["FT%"])][["Player", "FTA"]]

,Player,FTA
10,A.J. Bramlett,0.0
17,A.J. Guyton,0.0
52,Aaron Henry,0.0
69,Aaron McKie,0.0
100,Adam Flagler,0.0
...,...,...
13842,Yinka Dare,0.0
13873,Zach Lofton,0.0
13920,Zhou Qi,0.0
13944,Zylan Cheatham,0.0


In [235]:
stats.fillna(0, inplace=True)

In [236]:
stats.isnull().sum()

Player     0
Age        0
Team       0
Pos        0
G          0
GS         0
MP         0
FG         0
FGA        0
FG%        0
3P         0
3PA        0
3P%        0
2P         0
2PA        0
2P%        0
eFG%       0
FT         0
FTA        0
FT%        0
ORB        0
DRB        0
TRB        0
AST        0
STL        0
BLK        0
TOV        0
PF         0
PTS        0
Year       0
Pts Won    0
Pts Max    0
Share      0
W          0
L          0
W/L%       0
GB         0
PS/G       0
PA/G       0
SRS        0
dtype: int64

In [237]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS']

In [238]:
train = stats[stats["Year"] < 2024]

In [239]:
test = stats[stats["Year"] == 2024]

In [240]:
reg = Ridge(alpha=.1)

In [241]:
reg.fit(train[predictors], train["Share"])

Ridge(alpha=0.1)

In [242]:
predictions = reg.predict(test[predictors])

In [243]:
predictions = pd.DataFrame(predictions, columns=['predictions'], index=test.index)

In [244]:
predictions

,predictions
14,0.000428
19,-0.002547
27,-0.027402
43,0.025629
57,-0.011354
...,...
13872,0.021710
13891,-0.042060
13911,-0.004698
13923,-0.013201


In [245]:
combination = pd.concat([test[["Player", "Share"]],predictions], axis=1 )

In [246]:
combination

,Player,Share,predictions
14,A.J. Green,0.0,0.000428
19,A.J. Lawson,0.0,-0.002547
27,AJ Griffin,0.0,-0.027402
43,Aaron Gordon,0.0,0.025629
57,Aaron Holiday,0.0,-0.011354
...,...,...,...
13872,Zach LaVine,0.0,0.021710
13891,Zavier Simpson,0.0,-0.042060
13911,Zeke Nnaji,0.0,-0.004698
13923,Ziaire Williams,0.0,-0.013201


In [247]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions
11946,Shai Gilgeous-Alexander,0.646,0.166633
4960,Giannis Antetokounmpo,0.194,0.207000
5788,Jalen Brunson,0.143,0.098272
6344,Jayson Tatum,0.087,0.112347
723,Anthony Edwards,0.018,0.088158
3842,Domantas Sabonis,0.003,0.086366
7792,Kevin Durant,0.001,0.098908
8551,Lonnie Walker IV,0.000,0.005585
8494,Lester Quiñones,0.000,-0.003889
8504,Lindell Wigginton,0.000,-0.005676


In [248]:
mean_squared_error(combination["Share"], combination["predictions"])

0.0014284360575870332

In [249]:
combination["Share"].value_counts()

Share
0.000    487
0.018      1
0.003      1
0.194      1
0.143      1
0.087      1
0.001      1
0.646      1
Name: count, dtype: int64

In [250]:
combination = combination.sort_values("Share", ascending=False)
combination["Rk"] = list(range(1, combination.shape[0]+1))

In [251]:
combination.head(10)

,Player,Share,predictions,Rk
11946,Shai Gilgeous-Alexander,0.646,0.166633,1
4960,Giannis Antetokounmpo,0.194,0.207000,2
5788,Jalen Brunson,0.143,0.098272,3
6344,Jayson Tatum,0.087,0.112347,4
723,Anthony Edwards,0.018,0.088158,5
3842,Domantas Sabonis,0.003,0.086366,6
7792,Kevin Durant,0.001,0.098908,7
8551,Lonnie Walker IV,0.000,0.005585,8
8494,Lester Quiñones,0.000,-0.003889,9
8504,Lindell Wigginton,0.000,-0.005676,10


In [252]:
combination = combination.sort_values("predictions", ascending=False)
combination["Predicted_Rk"] = list(range(1, combination.shape[0]+1))

In [253]:
combination.head(10)

,Player,Share,predictions,Rk,Predicted_Rk
4960,Giannis Antetokounmpo,0.194,0.207000,2,1
6815,Joel Embiid,0.000,0.203548,93,2
8684,Luka Dončić,0.000,0.184096,15,3
11946,Shai Gilgeous-Alexander,0.646,0.166633,1,4
10170,Nikola Jokić,0.000,0.162689,203,5
719,Anthony Davis,0.000,0.130978,318,6
8440,LeBron James,0.000,0.123693,21,7
6344,Jayson Tatum,0.087,0.112347,4,8
13459,Victor Wembanyama,0.000,0.104163,173,9
7792,Kevin Durant,0.001,0.098908,7,10


In [254]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions,Rk,Predicted_Rk
11946,Shai Gilgeous-Alexander,0.646,0.166633,1,4
4960,Giannis Antetokounmpo,0.194,0.207000,2,1
5788,Jalen Brunson,0.143,0.098272,3,11
6344,Jayson Tatum,0.087,0.112347,4,8
723,Anthony Edwards,0.018,0.088158,5,16
3842,Domantas Sabonis,0.003,0.086366,6,17
7792,Kevin Durant,0.001,0.098908,7,10
11929,Seth Lundy,0.000,-0.011276,153,340
9401,Max Strus,0.000,-0.010396,48,333
5754,Jake LaRavia,0.000,-0.010019,389,329


In [255]:
def find_ap(combination):
    actual = combination.sort_values("Share", ascending=False).head(5)
    predicted = combination.sort_values("predictions", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index,row in predicted.iterrows():
        if row["Player"] in actual["Player"].values:
            found += 1
            ps.append(found / seen)
        seen += 1

    return sum(ps) / len(ps)

In [256]:
find_ap(combination)

0.5102272727272728

In [257]:
years = list(range(1991, 2024))

In [258]:
aps = []
all_predictions = []
for year in years[5:]:
    train = stats[stats["Year"] < year]
    test = stats[stats["Year"] == year]
    reg.fit(train[predictors],train["Share"])
    predictions = reg.predict(test[predictors])
    predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
    combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
    all_predictions.append(combination)
    aps.append(find_ap(combination))

In [259]:
sum(aps) / len(aps)

0.7097663105750387

In [260]:
def add_ranks(predictions):
    predictions = predictions.sort_values("predictions", ascending=False)
    predictions["Predicted_Rk"] = list(range(1,predictions.shape[0]+1))
    predictions = predictions.sort_values("Share", ascending=False)
    predictions["Rk"] = list(range(1,predictions.shape[0]+1))
    predictions["Diff"] = (predictions["Rk"] - predictions["Predicted_Rk"])
    return predictions

In [261]:
ranking = add_ranks(all_predictions[1])
ranking[ranking["Rk"] < 5].sort_values("Diff", ascending=False)

,Player,Share,predictions,Predicted_Rk,Rk,Diff
7470,Karl Malone,0.857,0.197240,2,1,-1
9555,Michael Jordan,0.832,0.174465,3,2,-1
5054,Grant Hill,0.327,0.130700,7,3,-4
12751,Tim Hardaway,0.207,0.064258,19,4,-15
